In [1]:
import json
from pathlib import Path
import pandas as pd 

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../tmp/musique-mini-temperature-sweep/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

860 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/musique',
   'name': 'answerable',
   'split': 'validation'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.1,
   'system_prompt': 'no-role.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'empty.json'},
  'run': 1},
 'metrics': {}}

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (df["params.dataset.path"] == "bdsaglam/musique-mini") & (
    df["params.qa.system_prompt"] != "no-role.txt"
)
df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

392 experiments after preprocessing


In [4]:
def parse_qa_technique(system_prompt_filename):
    if "cte" in system_prompt_filename:
        return "CTE"
    if "cot" in system_prompt_filename:
        return "COT"
    return "Standard"

df["params.qa.technique"] = df["params.qa.system_prompt"].map(parse_qa_technique)

In [5]:
import re

def parse_n_shot(x):
    if 'empty' in x:
        return 0
    # match the digit in the string
    x = re.search(r'\d+', x).group()
    return int(x)

assert parse_n_shot('empty.json') == 0
assert parse_n_shot('cot-2-shot.json') == 2

In [6]:
df['params.qa.n_shot'] = df['params.qa.few_shot_examples'].map(parse_n_shot)

In [7]:
def remove_ext(filename):
    return filename.replace(".txt", "")

df["params.qa.system_prompt"] = df["params.qa.system_prompt"].map(remove_ext)
df["params.qa.user_prompt_template"] = df["params.qa.user_prompt_template"].map(remove_ext)

In [8]:
def sorted_tuple(x):
    return tuple(sorted(x))

run_counts = (
    df.groupby(["params.qa.system_prompt", "params.qa.user_prompt_template", "params.qa.few_shot_examples", "params.qa.temperature"])[
        "params.run"
    ]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) < 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.temperature,params.run
72,standard/excellent-few,cq,standard-2-shot.json,0.0,"(2, 3)"


In [9]:
run_counts.loc[run_counts["params.run"].map(len) > 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.temperature,params.run


In [10]:
for col in run_counts.columns:
    print(f"# {col}")
    print(run_counts[col].unique())

# params.qa.system_prompt
['cot/excellent-format-reasoning' 'cot/format-minimal'
 'cot/format-reasoning' 'cot/format-think' 'cte/excellent-format-few'
 'cte/excellent-format-fewest' 'cte/format-few' 'cte/format-relevant-few'
 'standard/excellent-few' 'standard/excellent-fewest'
 'standard/helpful-output-format-few' 'standard/minimal'
 'standard/minimal-output-format' 'standard/minimal-output-format-few'
 'standard/minimal-output-format-few-no-prio'
 'standard/minimal-output-format-fewest']
# params.qa.user_prompt_template
['cq']
# params.qa.few_shot_examples
['empty.json' 'standard-2-shot.json']
# params.qa.temperature
[0.  0.1 0.3 0.5 0.7 1.  1.5 2. ]
# params.run
[(1, 2, 3) (2, 3)]


In [11]:
df.to_json('experiment-results.jsonl', orient='records', lines=True)